## Seq2Seq Classification Tutorial

### This tutorial is adapted from the PyTorch tutorial on ['Translation with a Sequence to Sequence Network and Attention'](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

def normalizeWindowsString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def randomTrainingExample(pairs, labels):
    rand_key = random.choice(list(pairs))
    return pairs[rand_key], labels[rand_key]


MAX_RAW_LENGTH = 30
MAX_RNN_LENGTH = 10
all_categories = []


def stripWords2MaxLen(sent):
    allwords = sent.split(' ')
    if(len(allwords) < MAX_RNN_LENGTH):
        return sent
    sent = ' '.join(allwords[i] for i in range(MAX_RNN_LENGTH-1))
    #print(sent)
    return sent

def filterSentencePair(sent0, sent1):
    return len(sent0.split(' ')) < MAX_RAW_LENGTH and \
        len(sent1.split(' ')) < MAX_RAW_LENGTH 

def readClassificationData(lang1):
    print("Reading Lines...")

    # Read the file and split into lines
    #with open('data/dev_w_id.txt', 'rb') as f:
    #  lines = f.readlines()#.strip().split('\n')
    #print(lines[0])
    #lines = open('data/dev_w_id.txt').read().strip().split('\n')
    lines = open('data/train_w_id.txt', encoding='windows-1252').readlines()
    print(len(lines))
    
    input_lang = Lang(lang1)
    pairs = {}
    labels = {}
    # Split every line into id, query, passage, label
    for l in lines:
        [id, sent0, sent1, label] = l.strip().replace('\n','').split('\t')
        
        if(filterSentencePair(sent0, sent1)):
            sent0 = stripWords2MaxLen(normalizeString(sent0))
            sent1 = stripWords2MaxLen(normalizeString(sent1))
            #print([id, normalizeWindowsString(sent0), normalizeWindowsString(sent1), label])
            pairs[id] = ([(sent0), (sent1)])
            labels[id] = (label)
            if label not in all_categories:
                all_categories.append((label))
            input_lang.addSentence((sent0))
            input_lang.addSentence((sent1))
    return input_lang, pairs, labels

def prepareClassificationData(lang1):
    input_lang, pairs, labels = readClassificationData(lang1)
    if( len(pairs) != len(labels)):
        print("Number of sentence pairs is not equal to label count")
        # exit(0)
    print("Read %s sentence pairs and labels" % len(pairs))

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    return input_lang, pairs, labels

input_lang, pairs, labels = prepareClassificationData('eng')
print(randomTrainingExample(pairs, labels))
print(randomTrainingExample(pairs, labels))
print(all_categories)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, class_size, dropout_p=0.1, max_length=MAX_RNN_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.class_size = class_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.hidden2class = nn.Linear(self.hidden_size, self.class_size)

    def forward(self, input, hidden, encoder_outputs):
        #print(input)
        embedded = self.embedding(input).view(1, 1, -1)
        #print(embedded.size())
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        
        class_out = F.log_softmax(self.hidden2class(hidden[0]), dim=1)
        return output, hidden, attn_weights, class_out

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    #print(indexes)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    #print(pair)
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(input_lang, pair[1])
    return (input_tensor, target_tensor)

def tensorFromLabel(category):
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    return category_tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

In [ ]:
#teacher_forcing_ratio = 1.0


def train(input_tensor, target_tensor, category_tensor, encoder, decoder, encoder_optimizer, \
          decoder_optimizer, criterion, max_length=MAX_RNN_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Teacher forcing: Feed the target as the next input
    final_decode_class = []
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention, decoder_class = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        #final_decode_class = decoder_class
        #loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing
    
    loss = criterion(decoder_class, category_tensor.cuda())
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()# / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    #[training_pairs,training_labels] = [randomTrainingExample(pairs, labels)
    #                  for i in range(n_iters)]
    training_pairs = []
    training_labels = []
    for i in range(n_iters):
        training_pair, training_label = randomTrainingExample(pairs, labels)
        training_pairs.append(training_pair)
        training_labels.append(training_label)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor, target_tensor = tensorsFromPair(training_pair)
        category_tensor = tensorFromLabel(training_labels[iter-1]).cuda()
        
        #print(input_tensor.size())
        #print(target_tensor.size())
        #print(category_tensor.size())
        #print()

        loss = train(input_tensor, target_tensor, category_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Training and Evaluating
=======================

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).cuda()
attn_decoder1 = AttnDecoderRNN(hidden_size, input_lang.n_words, len(all_categories), dropout_p=0.1).cuda()

trainIters(encoder1, attn_decoder1.cuda(), 75000, print_every=5000)

In [ ]:
def evaluate(input_tensor, target_tensor, encoder, decoder, max_length=MAX_RNN_LENGTH):
    encoder_hidden = encoder.initHidden()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #final_decode_class = []
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention, decoder_class = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        #final_decode_class = decoder_class
        #loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing
    
    #loss = criterion(decoder_class, category_tensor)
    #loss.backward()

    #encoder_optimizer.step()
    #decoder_optimizer.step()

    return decoder_class

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        training_pair, training_label = randomTrainingExample(pairs, labels)
        input_tensor, target_tensor = tensorsFromPair(training_pair)
        decoded_class = evaluate(input_tensor, target_tensor, encoder, decoder)
        guess, guess_i = categoryFromOutput(decoded_class)
        
        print(training_label, guess)
        print('')

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)